# Spoofed Speech Detection via Maximum Likelihood Estimation of Gaussian Mixture Models
The goal of synthetic speech detection is to determine whether a speech segment $S$ is natural or synthetic/converted speeach.

This notebook implements a Gaussian mixture model maximum likelihood (GMM-ML) classifier for synthetic (spoofed) speech detection. This approach uses regular mel frequency cepstral coefficients (MFCC) features and gives the best performance on the [ASVspoof 2015 dataset](https://www.idiap.ch/dataset/avspoof) among the standard classifiers (GMM-SV, GMM-UBM, ...). For more background information see: *Hanilçi, Cemal, Tomi Kinnunen, Md Sahidullah, and Aleksandr Sizov. "Classifiers for synthetic speech detection: a comparison." In INTERSPEECH 2015*. The scripts use the Python package [Bob.Bio.SPEAR 2.04](https://pypi.python.org/pypi/bob.bio.spear/2.0.4) for speaker recogntion.

This work is part of the ["DDoS Resilient Emergency Dispatch Center"](https://www.dhs.gov/science-and-technology/news/2015/09/04/dhs-st-awards-university-houston-26m-cyber-security-research) project at the University of Houston, funded by the Department of Homeland Security (DHS).


April 19, 2015

Lorenzo Rossi

(lorenzo **[dot]** rossi **[at]** gmail **[dot]** com)

In [70]:
import os
import time
import numpy as np
import pandas as pd
import scipy
from bob.bio.spear import preprocessor, extractor
from bob.bio.gmm import algorithm
from bob.io.base import HDF5File
from bob.learn import em
from sklearn.metrics import classification_report, roc_curve, roc_auc_score


def read(filename):
  """Read video.FrameContainer containing preprocessed frames"""

  fileName, fileExtension = os.path.splitext(filename)
  wav_filename = filename
  rate, data = scipy.io.wavfile.read(str(wav_filename)) # the data is read in its native format
  if data.dtype =='int16':
    data = np.cast['float'](data)
  return [rate,data]


WAV_FOLDER = '/data/chendan/wav/' #'ASV2015dataset/wav/' # Path to folder containing speakers .wav subfolders
LABEL_FOLDER = '/data/chendan/protocal/CM_protocol/' #'ASV2015dataset/CM_protocol/' # Path to ground truth csv files

EXT = '.wav'
%matplotlib inline

## Loading the Ground Truth
Load the dataframes (tables) with the labels for the training, development and evaluation (hold out) sets. Each subfolder corresponds to a different speaker. For example, T1 and D4 indicate the subfolders associated to the utterances and spoofed segments of speakers T1 and D4, respectively in training and development sets. Note that number of evaluation samples >> number of development samples >> testing samples.

You can either select the speakers in each set one by one, *e.g.*:
```
train_subfls = ['T1', 'T2']
``` 
will only load segments from speakers T1 and T2 for training,

or use all the available speakers in a certain subset by leaving the list empty, *e.g.*:
```
devel_subfls = [] 
```
will load all the available Dx speaker segments for the development stage. If you are running this notebook for the first time, you may want to start only with 2 or so speakers per set for sake of quick testing. All the scripts may take several hours to run on the full size datsets.

In [86]:
train_subfls = ['T1','T3']#, 'T2', 'T3', 'T4', 'T5', 'T6', 'T7', 'T8', 'T9', 'T13']  #T13 used instead of T10 for gender balance
devel_subfls = ['D1','D3']#, 'D2', 'D3', 'D4', 'D5', 'D6', 'D7', 'D8', 'D9', 'D10']
evalu_subfls = ['E1']#, 'E2', 'E3', 'E4', 'E5', 'E6','E7',  'E8', 'E9', 'E10']
train = pd.read_csv(LABEL_FOLDER + 'cm_train.trn', sep=' ', header=None, names=['folder','file','method','source'])
if len(train_subfls): train = train[train.folder.isin(train_subfls)]
train.sort_values(['folder', 'file'], inplace=True)
# print(train)
devel = pd.read_csv(LABEL_FOLDER + 'cm_develop.ndx', sep=' ', header=None, names=['folder','file','method','source'])
if len(devel_subfls): devel = devel[devel.folder.isin(devel_subfls)]
devel.sort_values(['folder', 'file'], inplace=True)
print(devel)
evalu = pd.read_csv(LABEL_FOLDER +'cm_evaluation_0103.ndx', sep=' ', header=None, names=['folder','file','method','source'])
if len(evalu_subfls): evalu = evalu[evalu.folder.isin(evalu_subfls)]

evalu.sort_values(['folder', 'file'], inplace=True)

label_2_class = {'human':1, 'spoof':0}

print('training samples:',len(train))
print('development samples:',len(devel))
print('evaluation samples:',len(evalu))

     folder        file method source
2597     D1  D1_1002598  human  human
2598     D1  D1_1002599  human  human
2599     D1  D1_1002600  human  human
2600     D1  D1_1002601  human  human
2601     D1  D1_1002602  human  human
...     ...         ...    ...    ...
7767     D3  D3_1007768     S1  spoof
7768     D3  D3_1007769     S2  spoof
7769     D3  D3_1007770     S3  spoof
7770     D3  D3_1007771     S4  spoof
7771     D3  D3_1007772     S5  spoof

[3050 rows x 4 columns]
training samples: 1310
development samples: 3050
evaluation samples: 4185


## Speech Preprocessing and MFCC Extraction
Silence removal and MFCC feature extraction for training segments. More details about the bob.bio.spear involved libraries at:
https://www.idiap.ch/software/bob/docs/latest/bioidiap/bob.bio.spear/master/implemented.html

You can also skip this stage and load a set of feaures (see **Loading features** cell).

In [72]:
# Parameters
n_ceps = 60 # number of ceptral coefficients (implicit in extractor)
silence_removal_ratio = .1

In [73]:
subfolders = train_subfls
ground_truth = train

# initialize feature matrix
features = []
y = np.zeros((len(ground_truth),))
print("Extracting features for training stage.")

vad = preprocessor.Energy_Thr(ratio_threshold=silence_removal_ratio)
cepstrum = extractor.Cepstral()

k = 0
start_time = time.clock()

n_subfls = len(subfolders)
for folder in subfolders[0:n_subfls]:
    print(folder, end=", ")
    folder = "".join((WAV_FOLDER,folder,'/'))
    # print(folder)
    f_list = os.listdir(folder)
    # print(f_list)
    for f_name in f_list:
        # ground truth
        try: 
            # 根据train中file(路径)字段，提取相应.wav文件的source(spoof or human)字段
            label = ground_truth[ground_truth.file==f_name[:-len(EXT)]].source.values[0]
        except IndexError:
            continue
        y[k] = label_2_class[label]
        # silence removal
        # print(f_name)
        # print(vad)
        x = read(folder+f_name)
        vad_data = vad(x)

        if vad_data:
            if not vad_data[2].max():
                vad = preprocessor.Energy_Thr(ratio_threshold=silence_removal_ratio*.8)
                vad_data = vad(x)
                vad = preprocessor.Energy_Thr(ratio_threshold=silence_removal_ratio)
            # MFCC extraction 
            mfcc = cepstrum(vad_data)
            features.append(mfcc)
            k += 1
        else :
            features.append(None)

Xf = np.array(features)
print(k,"files processed in",(time.clock()-start_time)/60,"minutes.")

Extracting features for training stage.
T1, bob.bio.spear@2020-01-03 13:09:30,422 -- INFO: After thresholded Energy-based VAD there are 209 frames remaining over 407
bob.bio.spear@2020-01-03 13:09:30,528 -- INFO: After thresholded Energy-based VAD there are 143 frames remaining over 273
bob.bio.spear@2020-01-03 13:09:30,601 -- INFO: After thresholded Energy-based VAD there are 140 frames remaining over 460


/home/sundata/anaconda3/envs/bob_py3/lib/python3.7/site-packages/ipykernel_launcher.py:13: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  del sys.path[0]


bob.bio.spear@2020-01-03 13:09:30,676 -- INFO: After thresholded Energy-based VAD there are 194 frames remaining over 351
bob.bio.spear@2020-01-03 13:09:30,774 -- INFO: After thresholded Energy-based VAD there are 116 frames remaining over 288
bob.bio.spear@2020-01-03 13:09:30,835 -- INFO: After thresholded Energy-based VAD there are 176 frames remaining over 411
bob.bio.spear@2020-01-03 13:09:30,925 -- INFO: After thresholded Energy-based VAD there are 93 frames remaining over 245
bob.bio.spear@2020-01-03 13:09:30,974 -- INFO: After thresholded Energy-based VAD there are 117 frames remaining over 249
bob.bio.spear@2020-01-03 13:09:31,034 -- INFO: After thresholded Energy-based VAD there are 140 frames remaining over 231
bob.bio.spear@2020-01-03 13:09:31,104 -- INFO: After thresholded Energy-based VAD there are 126 frames remaining over 287
bob.bio.spear@2020-01-03 13:09:31,169 -- INFO: After thresholded Energy-based VAD there are 183 frames remaining over 407
bob.bio.spear@2020-01-03 

bob.bio.spear@2020-01-03 13:09:35,280 -- INFO: After thresholded Energy-based VAD there are 173 frames remaining over 485
bob.bio.spear@2020-01-03 13:09:35,370 -- INFO: After thresholded Energy-based VAD there are 244 frames remaining over 454
bob.bio.spear@2020-01-03 13:09:35,492 -- INFO: After thresholded Energy-based VAD there are 145 frames remaining over 245
bob.bio.spear@2020-01-03 13:09:35,566 -- INFO: After thresholded Energy-based VAD there are 119 frames remaining over 315
bob.bio.spear@2020-01-03 13:09:35,628 -- INFO: After thresholded Energy-based VAD there are 108 frames remaining over 219
bob.bio.spear@2020-01-03 13:09:35,683 -- INFO: After thresholded Energy-based VAD there are 93 frames remaining over 332
bob.bio.spear@2020-01-03 13:09:35,733 -- INFO: After thresholded Energy-based VAD there are 112 frames remaining over 228
bob.bio.spear@2020-01-03 13:09:35,792 -- INFO: After thresholded Energy-based VAD there are 208 frames remaining over 402
bob.bio.spear@2020-01-03 

bob.bio.spear@2020-01-03 13:09:40,452 -- INFO: After thresholded Energy-based VAD there are 142 frames remaining over 451
bob.bio.spear@2020-01-03 13:09:40,526 -- INFO: After thresholded Energy-based VAD there are 115 frames remaining over 208
bob.bio.spear@2020-01-03 13:09:40,585 -- INFO: After thresholded Energy-based VAD there are 79 frames remaining over 211
bob.bio.spear@2020-01-03 13:09:40,628 -- INFO: After thresholded Energy-based VAD there are 222 frames remaining over 523
bob.bio.spear@2020-01-03 13:09:40,740 -- INFO: After thresholded Energy-based VAD there are 141 frames remaining over 304
bob.bio.spear@2020-01-03 13:09:40,812 -- INFO: After thresholded Energy-based VAD there are 137 frames remaining over 262
bob.bio.spear@2020-01-03 13:09:40,881 -- INFO: After thresholded Energy-based VAD there are 89 frames remaining over 241
bob.bio.spear@2020-01-03 13:09:40,928 -- INFO: After thresholded Energy-based VAD there are 113 frames remaining over 223
bob.bio.spear@2020-01-03 1

...

bob.bio.spear@2020-01-03 13:10:54,188 -- INFO: After thresholded Energy-based VAD there are 83 frames remaining over 182
bob.bio.spear@2020-01-03 13:10:54,233 -- INFO: After thresholded Energy-based VAD there are 197 frames remaining over 450
bob.bio.spear@2020-01-03 13:10:54,333 -- INFO: After thresholded Energy-based VAD there are 109 frames remaining over 246
bob.bio.spear@2020-01-03 13:10:54,391 -- INFO: After thresholded Energy-based VAD there are 189 frames remaining over 365
bob.bio.spear@2020-01-03 13:10:54,485 -- INFO: After thresholded Energy-based VAD there are 101 frames remaining over 203
bob.bio.spear@2020-01-03 13:10:54,538 -- INFO: After thresholded Energy-based VAD there are 216 frames remaining over 347
bob.bio.spear@2020-01-03 13:10:54,646 -- INFO: After thresholded Energy-based VAD there are 245 frames remaining over 523
bob.bio.spear@2020-01-03 13:10:54,770 -- INFO: After thresholded Energy-based VAD there are 137 frames remaining over 377
bob.bio.spear@2020-01-03 

bob.bio.spear@2020-01-03 13:10:59,535 -- INFO: After thresholded Energy-based VAD there are 134 frames remaining over 279
bob.bio.spear@2020-01-03 13:10:59,604 -- INFO: After thresholded Energy-based VAD there are 123 frames remaining over 254
bob.bio.spear@2020-01-03 13:10:59,668 -- INFO: After thresholded Energy-based VAD there are 209 frames remaining over 382
bob.bio.spear@2020-01-03 13:10:59,772 -- INFO: After thresholded Energy-based VAD there are 134 frames remaining over 261
bob.bio.spear@2020-01-03 13:10:59,839 -- INFO: After thresholded Energy-based VAD there are 138 frames remaining over 208
bob.bio.spear@2020-01-03 13:10:59,910 -- INFO: After thresholded Energy-based VAD there are 255 frames remaining over 484
bob.bio.spear@2020-01-03 13:11:00,042 -- INFO: After thresholded Energy-based VAD there are 157 frames remaining over 263
bob.bio.spear@2020-01-03 13:11:00,123 -- INFO: After thresholded Energy-based VAD there are 197 frames remaining over 385
bob.bio.spear@2020-01-03

bob.bio.spear@2020-01-03 13:11:04,763 -- INFO: After thresholded Energy-based VAD there are 96 frames remaining over 238
bob.bio.spear@2020-01-03 13:11:04,815 -- INFO: After thresholded Energy-based VAD there are 136 frames remaining over 420
bob.bio.spear@2020-01-03 13:11:04,887 -- INFO: After thresholded Energy-based VAD there are 201 frames remaining over 334
bob.bio.spear@2020-01-03 13:11:04,989 -- INFO: After thresholded Energy-based VAD there are 135 frames remaining over 297
bob.bio.spear@2020-01-03 13:11:05,057 -- INFO: After thresholded Energy-based VAD there are 171 frames remaining over 291
bob.bio.spear@2020-01-03 13:11:05,143 -- INFO: After thresholded Energy-based VAD there are 109 frames remaining over 274
bob.bio.spear@2020-01-03 13:11:05,199 -- INFO: After thresholded Energy-based VAD there are 168 frames remaining over 343
bob.bio.spear@2020-01-03 13:11:05,284 -- INFO: After thresholded Energy-based VAD there are 158 frames remaining over 445
bob.bio.spear@2020-01-03 

bob.bio.spear@2020-01-03 13:11:09,797 -- INFO: After thresholded Energy-based VAD there are 322 frames remaining over 518
bob.bio.spear@2020-01-03 13:11:09,954 -- INFO: After thresholded Energy-based VAD there are 160 frames remaining over 257
bob.bio.spear@2020-01-03 13:11:10,033 -- INFO: After thresholded Energy-based VAD there are 65 frames remaining over 228
bob.bio.spear@2020-01-03 13:11:10,070 -- INFO: After thresholded Energy-based VAD there are 245 frames remaining over 436
bob.bio.spear@2020-01-03 13:11:10,190 -- INFO: After thresholded Energy-based VAD there are 108 frames remaining over 184
bob.bio.spear@2020-01-03 13:11:10,246 -- INFO: After thresholded Energy-based VAD there are 240 frames remaining over 441
bob.bio.spear@2020-01-03 13:11:10,369 -- INFO: After thresholded Energy-based VAD there are 132 frames remaining over 261
bob.bio.spear@2020-01-03 13:11:10,435 -- INFO: After thresholded Energy-based VAD there are 48 frames remaining over 131
bob.bio.spear@2020-01-03 1

/home/sundata/anaconda3/envs/bob_py3/lib/python3.7/site-packages/ipykernel_launcher.py:49: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead


In [74]:
print(Xf[y==0].shape)

(1010,)


### Saving features

In [25]:
np.save('X1.npy',Xf)
np.save('y1.npy',y)
print('Feature and label matrices saved to disk')

Feature and label matrices saved to disk


### Loading features

In [9]:
# Load already extracter features to skip the preprocessing-extraction stage
# Xf = np.load('train_features_10.npy')
# y = np.load('y_10.npy')
Xf = np.load('X.npy',allow_pickle=True)
y = np.load('y.npy',allow_pickle=True)

In [23]:
print(Xf.shape)

(655,)


# GMM - ML Classification

## GMM Training
Train the GMMs for natural and synthetic speach. For documentation on bob.bio k-means and GMM machines see:
https://pythonhosted.org/bob.learn.em/guide.html

You can also skip the training stage and load an already trained GMM model (see cell **Loading GMM Model**).

In [75]:
# Parameters of the GMM machines
n_gaussians = 128 # number of Gaussians
max_iterats = 25 # maximum number of iterations

### GMM for natural speech 

In [76]:
# Initialize and train k-means machine: the means will initialize EM algorithm for GMM machine
start_time = time.clock()
kmeans_nat = em.KMeansMachine(n_gaussians,n_ceps)
kmeansTrainer = em.KMeansTrainer()
print(kmeans_nat.shape,Xf[y==1].shape)
em.train(kmeansTrainer, kmeans_nat, np.vstack(Xf[y==1]), max_iterations = max_iterats, convergence_threshold = 1e-5)
#kmeans_nat.means

# initialize and train GMM machine
gmm_nat = em.GMMMachine(n_gaussians,n_ceps)
trainer = em.ML_GMMTrainer(True, True, True)
gmm_nat.means = kmeans_nat.means
em.train(trainer, gmm_nat, np.vstack(Xf[y==1]), max_iterations = max_iterats, convergence_threshold = 1e-5)
#gmm_nat.save(HDF5File('gmm_nat.hdf5', 'w'))
print("Done in:", (time.clock() - start_time)/60, "minutes")
print(gmm_nat)

/home/sundata/anaconda3/envs/bob_py3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  


(128, 60) (300,)
Done in: 0.7205086166666661 minutes


/home/sundata/anaconda3/envs/bob_py3/lib/python3.7/site-packages/ipykernel_launcher.py:15: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  from ipykernel import kernelapp as app


### GMM for synthetic speech

In [77]:
# initialize and train k-means machine: the means will initialize EM algorithm for GMM machine
start_time = time.clock()
kmeans_synt = em.KMeansMachine(n_gaussians,n_ceps)
kmeansTrainer = em.KMeansTrainer()
print(kmeans_synt.shape,Xf[y==0].shape)
em.train(kmeansTrainer, kmeans_synt, np.vstack(Xf[y==0]), max_iterations = max_iterats, convergence_threshold = 1e-5)

# initialize and train GMM machine
gmm_synt = em.GMMMachine(n_gaussians,n_ceps)
trainer = em.ML_GMMTrainer(True, True, True)
gmm_synt.means = kmeans_synt.means
em.train(trainer, gmm_synt, np.vstack(Xf[y==0]), max_iterations = max_iterats, convergence_threshold = 1e-5)
print("Done in:", (time.clock() - start_time)/60, "minutes")
#gmm_synt.save(HDF5File('gmm_synt.hdf5', 'w'))
print(gmm_synt)

/home/sundata/anaconda3/envs/bob_py3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  


(128, 60) (1010,)
Done in: 2.4534923833333324 minutes


/home/sundata/anaconda3/envs/bob_py3/lib/python3.7/site-packages/ipykernel_launcher.py:13: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  del sys.path[0]


### Loading GMM model

In [4]:
gmm_nat = em.GMMMachine()
gmm_nat.load(HDF5File('gmm_nat.hdf5', 'r'))
gmm_synt = em.GMMMachine()
gmm_synt.load(HDF5File('gmm_synt.hdf5','r'))

In [ ]:
np.save('p_gmm_ml_eval_10.npy',llr_score)
np.save('z_gmm_ml_eval_est_10.npy',z_gmm)

## GMM-ML Scoring
Extract the features for the testing data, compute the likelihood ratio test and  compute ROC AUC and estimated EER scores.

In [87]:
status = 'evalu' # 'devel'(= test) OR 'evalu'(= hold out)
start_time = time.clock()

if status == 'devel':
    subfolders = devel_subfls
    ground_truth = devel
elif status == 'evalu':
    subfolders = evalu_subfls
    ground_truth = evalu
n_subfls = len(subfolders)
# initialize score and class arrays
llr_gmm_score = np.zeros(len(ground_truth),)
z_gmm = np.zeros(len(ground_truth),)
print(status)

vad = preprocessor.Energy_Thr(ratio_threshold=.1)
cepstrum = extractor.Cepstral()

k = 0
thr = .5
speaker_list = ground_truth.folder.unique()

for speaker_id in speaker_list:
    #speaker = ground_truth[ground_truth.folder==speaker_id]
    f_list = list(ground_truth[ground_truth.folder==speaker_id].file)
    folder = "".join([WAV_FOLDER,speaker_id,'/'])
    print(f_list)
    print(speaker_id, end=',')

    for f in f_list:
        f_name = "".join([folder,f,'.wav'])
        if os.path.exists(f_name):
            x = read(f_name)
            # voice activity detection
            vad_data = vad(x)
            if vad_data:
                if not vad_data[2].max():
                    vad = preprocessor.Energy_Thr(ratio_threshold=.08)
                    vad_data = vad(x)
                    vad = preprocessor.Energy_Thr(ratio_threshold=.1)
                # MFCC extraction 
                mfcc = cepstrum(vad_data)
                # Log likelihood ratio computation
                llr_gmm_score[k] = gmm_nat(mfcc)-gmm_synt(mfcc)
                z_gmm[k] = int(llr_gmm_score[k]>0)
                k += 1
        
ground_truth['z'] = ground_truth.source.map(lambda x: int(x=='human'))
ground_truth['z_gmm'] = z_gmm
ground_truth['score_gmm'] = llr_gmm_score
#print(roc_auc_score(ground_truth.z, ground_truth.z_gmm))
print(k,"files processed in",(time.clock()-start_time)/60,"minutes.")

evalu
['E10000034', 'E10000043', 'E10000087', 'E10000142', 'E10000143', 'E10000164', 'E10000195', 'E10000515', 'E10000516', 'E10000543', 'E10000637', 'E10000720', 'E10000748', 'E10000796', 'E10000831', 'E10000862', 'E10000870', 'E10000875', 'E10000953', 'E10001183', 'E10001240', 'E10001295', 'E10001297', 'E10001305', 'E10001312', 'E10001355', 'E10001372', 'E10001446', 'E10001460', 'E10001605', 'E10001663', 'E10001719', 'E10001818', 'E10001937', 'E10001953', 'E10001955', 'E10001972', 'E10002045', 'E10002164', 'E10002198', 'E10002250', 'E10002310', 'E10002316', 'E10002379', 'E10002447', 'E10002605', 'E10002659', 'E10002676', 'E10002713', 'E10002736', 'E10002757', 'E10002789', 'E10002830', 'E10002835', 'E10002879', 'E10002936', 'E10003011', 'E10003059', 'E10003148', 'E10003151', 'E10003189', 'E10003193', 'E10003209', 'E10003305', 'E10003316', 'E10003328', 'E10003331', 'E10003334', 'E10003396', 'E10003575', 'E10003590', 'E10003592', 'E10003656', 'E10003660', 'E10003679', 'E10003688', 'E100

bob.bio.spear@2020-01-03 15:35:21,038 -- INFO: After thresholded Energy-based VAD there are 84 frames remaining over 228
bob.bio.spear@2020-01-03 15:35:21,083 -- INFO: After thresholded Energy-based VAD there are 84 frames remaining over 195
bob.bio.spear@2020-01-03 15:35:21,129 -- INFO: After thresholded Energy-based VAD there are 113 frames remaining over 298


/home/sundata/anaconda3/envs/bob_py3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  


bob.bio.spear@2020-01-03 15:35:21,191 -- INFO: After thresholded Energy-based VAD there are 91 frames remaining over 317
bob.bio.spear@2020-01-03 15:35:21,241 -- INFO: After thresholded Energy-based VAD there are 94 frames remaining over 245
bob.bio.spear@2020-01-03 15:35:21,292 -- INFO: After thresholded Energy-based VAD there are 153 frames remaining over 305
bob.bio.spear@2020-01-03 15:35:21,372 -- INFO: After thresholded Energy-based VAD there are 136 frames remaining over 194
bob.bio.spear@2020-01-03 15:35:21,442 -- INFO: After thresholded Energy-based VAD there are 75 frames remaining over 246
bob.bio.spear@2020-01-03 15:35:21,485 -- INFO: After thresholded Energy-based VAD there are 197 frames remaining over 309
bob.bio.spear@2020-01-03 15:35:21,586 -- INFO: After thresholded Energy-based VAD there are 164 frames remaining over 347
bob.bio.spear@2020-01-03 15:35:21,671 -- INFO: After thresholded Energy-based VAD there are 118 frames remaining over 279
bob.bio.spear@2020-01-03 15

bob.bio.spear@2020-01-03 15:35:26,333 -- INFO: After thresholded Energy-based VAD there are 169 frames remaining over 208
bob.bio.spear@2020-01-03 15:35:26,419 -- INFO: After thresholded Energy-based VAD there are 115 frames remaining over 364
bob.bio.spear@2020-01-03 15:35:26,482 -- INFO: After thresholded Energy-based VAD there are 214 frames remaining over 416
bob.bio.spear@2020-01-03 15:35:26,592 -- INFO: After thresholded Energy-based VAD there are 84 frames remaining over 113
bob.bio.spear@2020-01-03 15:35:26,637 -- INFO: After thresholded Energy-based VAD there are 218 frames remaining over 429
bob.bio.spear@2020-01-03 15:35:26,751 -- INFO: After thresholded Energy-based VAD there are 290 frames remaining over 557
bob.bio.spear@2020-01-03 15:35:26,901 -- INFO: After thresholded Energy-based VAD there are 161 frames remaining over 467
bob.bio.spear@2020-01-03 15:35:26,987 -- INFO: After thresholded Energy-based VAD there are 96 frames remaining over 300
bob.bio.spear@2020-01-03 1

...

bob.bio.spear@2020-01-03 15:40:17,003 -- INFO: After thresholded Energy-based VAD there are 103 frames remaining over 203
bob.bio.spear@2020-01-03 15:40:17,058 -- INFO: After thresholded Energy-based VAD there are 127 frames remaining over 429
bob.bio.spear@2020-01-03 15:40:17,127 -- INFO: After thresholded Energy-based VAD there are 95 frames remaining over 244
bob.bio.spear@2020-01-03 15:40:17,179 -- INFO: After thresholded Energy-based VAD there are 221 frames remaining over 416
bob.bio.spear@2020-01-03 15:40:17,293 -- INFO: After thresholded Energy-based VAD there are 137 frames remaining over 214
bob.bio.spear@2020-01-03 15:40:17,363 -- INFO: After thresholded Energy-based VAD there are 116 frames remaining over 187
bob.bio.spear@2020-01-03 15:40:17,424 -- INFO: After thresholded Energy-based VAD there are 194 frames remaining over 309
bob.bio.spear@2020-01-03 15:40:17,524 -- INFO: After thresholded Energy-based VAD there are 136 frames remaining over 253
bob.bio.spear@2020-01-03 

bob.bio.spear@2020-01-03 15:40:22,232 -- INFO: After thresholded Energy-based VAD there are 85 frames remaining over 428
bob.bio.spear@2020-01-03 15:40:22,283 -- INFO: After thresholded Energy-based VAD there are 239 frames remaining over 565
bob.bio.spear@2020-01-03 15:40:22,409 -- INFO: After thresholded Energy-based VAD there are 113 frames remaining over 505
bob.bio.spear@2020-01-03 15:40:22,472 -- INFO: After thresholded Energy-based VAD there are 83 frames remaining over 197
bob.bio.spear@2020-01-03 15:40:22,518 -- INFO: After thresholded Energy-based VAD there are 232 frames remaining over 372
bob.bio.spear@2020-01-03 15:40:22,637 -- INFO: After thresholded Energy-based VAD there are 104 frames remaining over 302
bob.bio.spear@2020-01-03 15:40:22,693 -- INFO: After thresholded Energy-based VAD there are 138 frames remaining over 288
bob.bio.spear@2020-01-03 15:40:22,766 -- INFO: After thresholded Energy-based VAD there are 153 frames remaining over 437
bob.bio.spear@2020-01-03 1

bob.bio.spear@2020-01-03 15:40:27,335 -- INFO: After thresholded Energy-based VAD there are 238 frames remaining over 467
bob.bio.spear@2020-01-03 15:40:27,457 -- INFO: After thresholded Energy-based VAD there are 86 frames remaining over 441
bob.bio.spear@2020-01-03 15:40:27,507 -- INFO: After thresholded Energy-based VAD there are 92 frames remaining over 174
bob.bio.spear@2020-01-03 15:40:27,557 -- INFO: After thresholded Energy-based VAD there are 181 frames remaining over 411
bob.bio.spear@2020-01-03 15:40:27,652 -- INFO: After thresholded Energy-based VAD there are 168 frames remaining over 305
bob.bio.spear@2020-01-03 15:40:27,739 -- INFO: After thresholded Energy-based VAD there are 141 frames remaining over 205
bob.bio.spear@2020-01-03 15:40:27,812 -- INFO: After thresholded Energy-based VAD there are 145 frames remaining over 339
bob.bio.spear@2020-01-03 15:40:27,889 -- INFO: After thresholded Energy-based VAD there are 176 frames remaining over 280
bob.bio.spear@2020-01-03 1

bob.bio.spear@2020-01-03 15:40:32,508 -- INFO: After thresholded Energy-based VAD there are 189 frames remaining over 411
bob.bio.spear@2020-01-03 15:40:32,606 -- INFO: After thresholded Energy-based VAD there are 117 frames remaining over 206
bob.bio.spear@2020-01-03 15:40:32,667 -- INFO: After thresholded Energy-based VAD there are 65 frames remaining over 208
bob.bio.spear@2020-01-03 15:40:32,704 -- INFO: After thresholded Energy-based VAD there are 98 frames remaining over 249
bob.bio.spear@2020-01-03 15:40:32,756 -- INFO: After thresholded Energy-based VAD there are 65 frames remaining over 124
bob.bio.spear@2020-01-03 15:40:32,791 -- INFO: After thresholded Energy-based VAD there are 64 frames remaining over 199
bob.bio.spear@2020-01-03 15:40:32,827 -- INFO: After thresholded Energy-based VAD there are 139 frames remaining over 344
bob.bio.spear@2020-01-03 15:40:32,901 -- INFO: After thresholded Energy-based VAD there are 193 frames remaining over 437
bob.bio.spear@2020-01-03 15:

/home/sundata/anaconda3/envs/bob_py3/lib/python3.7/site-packages/ipykernel_launcher.py:52: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead


In [88]:
# Performance evaluation

#实际humans个数
humans = sum(ground_truth.z==1)
print("实际humans个数:", humans)

#实际spoofed个数
spoofed = sum(ground_truth.z==0)
print("实际spoofed个数:", spoofed)

#预测humans个数
pre_humans = sum(ground_truth.z_gmm==1)
print("预测humans个数:", pre_humans)

#预测spoofed个数
pre_spoofed = sum(ground_truth.z_gmm==0)
print("预测spoofed个数:", pre_spoofed)

tp = 0
tn = 0
fp = 0
fn = 0
for i in ground_truth.index:
    if ground_truth.z[i] == 1 and ground_truth.z_gmm[i]==1:
        tp = tp + 1
    elif ground_truth.z[i] == 1 and ground_truth.z_gmm[i]==0:
        fn = fn + 1
    elif ground_truth.z[i] == 0 and ground_truth.z_gmm[i]==0:
        tn = tn + 1
    elif ground_truth.z[i] == 0 and ground_truth.z_gmm[i]==1:
        fp = fp + 1
print("TP_Num:", tp)
print("TN_Num:", tn)
print("FP_Num:", fp)
print("FN_Num:", fn)

tpr = tp/(tp+fn)
fpr = fp/(fp+tn)
tnr = tn/(fp+tn)
fnr = fn/(tp+fn)
acc = (tp+tn)/(tp+fp+tn+fn)
print("TPR:", tpr)
print("FPR:", fpr)
print("TNR:", tnr)
print("FNR:", fnr)
print("Accuracy:", acc)

# humans = z_gmm[z_dvl==0]
# spoofed = z_gmm[z_dvl==1]
# fnr = 100*(1-(humans<thr).sum()/len(humans))
# fpr = 100*(1-(spoofed>=thr).sum()/len(spoofed))
# print("ROC AUC score:", roc_auc_score(z_dvl,z_gmm))
# print("False negative rate %:", fnr)
# print("False positive rate %:", fpr)
# print("EER %: <=", (fnr+fpr)/2)

实际humans个数: 185
实际spoofed个数: 4000
预测humans个数: 872
预测spoofed个数: 3313
TP_Num: 177
TN_Num: 3305
FP_Num: 695
FN_Num: 8
TPR: 0.9567567567567568
FPR: 0.17375
TNR: 0.82625
FNR: 0.043243243243243246
Accuracy: 0.8320191158900836


In [47]:
ground_truth

,folder,file,method,source,z,z_gmm,score_gmm
2597,D1,D1_1002598,human,human,1,1.0,1.447147
2598,D1,D1_1002599,human,human,1,1.0,2.336380
2599,D1,D1_1002600,human,human,1,1.0,3.082637
2600,D1,D1_1002601,human,human,1,1.0,3.716948
2601,D1,D1_1002602,human,human,1,1.0,0.914256
...,...,...,...,...,...,...,...
7767,D3,D3_1007768,S1,spoof,0,0.0,-2.641017
7768,D3,D3_1007769,S2,spoof,0,0.0,-0.571958
7769,D3,D3_1007770,S3,spoof,0,0.0,-5.441518
7770,D3,D3_1007771,S4,spoof,0,0.0,-4.762759


### EER computation

Adjust the threshold $thr$ to reduce $FNR-FPR$ for a more accurate estimate of the $EER$.

The Equal Error Rate ($EER$) is the value where the false negative rate ($FNR$) equals the false positive rate ($FPR$). It's an error metric commonly used to characterize biometric systems.

In [55]:
thr = -.115
pz = llr_gmm_score
spoofed = pz[np.array(ground_truth.z)==1]
humans = pz[np.array(ground_truth.z)==0]
fnr = 100*(humans>thr).sum()/len(humans)
fpr = 100*(spoofed<=thr).sum()/len(spoofed)
print("False negative vs positive rates %:", fnr, fpr)
print("FNR - FPR %:", fnr-fpr)
if np.abs(fnr-fpr) <.25:
    print("EER =", (fnr+fpr)/2,"%")
else:
    print("EER ~", (fnr+fpr)/2,"%")

False negative vs positive rates %: 4.701754385964913 10.0
FNR - FPR %: -5.298245614035087
EER ~ 7.350877192982456 %
